# Import Required Libraries
Import all necessary libraries, including pandas, numpy, sklearn, and others.

In [33]:
# Import necessary libraries
import pandas as pd
import numpy as np
# Model selection and evaluation
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV # Añadir RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
# Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor # Añadir KNeighborsRegressor
# Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Utilities
import sys
from scipy.stats import randint # Para RandomizedSearchCV

# Define Utility Functions
Define the functions `load_and_merge_data`, `create_preprocessor`, `preprocess_and_split`, and `train_predict_evaluate` in separate cells.

In [34]:
# Define the `load_and_merge_data` function (sin cambios)
def load_and_merge_data(features_path, labels_path):
    """
    Carga los datasets de características y etiquetas, y los une en un único DataFrame.
    """
    try:
        features_df = pd.read_csv(features_path, parse_dates=['week_start_date'])
        labels_df = pd.read_csv(labels_path)
        df = pd.merge(features_df, labels_df, on=['city', 'year', 'weekofyear'])
        # Ordenar cronológicamente puede ser mejor para evaluación, pero mantenemos el original por ahora
        df.sort_values(by=['city', 'week_start_date'], inplace=True)
        return df
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None

# Define the `create_preprocessor` function (sin cambios)
def create_preprocessor(df):
    """
    Crea un preprocesador para manejar características categóricas y numéricas.
    Incluye StandardScaler, bueno para KNN.
    """
    numeric_features = df.select_dtypes(include=np.number).columns.tolist()
    # Excluir columnas que no son features directas o el target si estuviera
    numeric_features = [col for col in numeric_features if col not in ['year', 'weekofyear', 'total_cases']]
    categorical_features = ['city']

    # Pipelines para transformar datos numéricos y categóricos
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')), # Usar mediana es más robusto
        ('scaler', StandardScaler()) # Importante para KNN
    ])
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # sparse_output=False para facilitar manejo
    ])

    # ColumnTransformer para aplicar las transformaciones
    # Usar set_output para que devuelva DataFrames
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough', # Mantiene columnas no especificadas (ej. month, day, day_of_week)
        verbose_feature_names_out=False # Nombres de columnas más limpios
    )
    preprocessor.set_output(transform="pandas")
    return preprocessor


# Define the `preprocess_and_split` function
def preprocess_and_split(df, preprocessor, target_col='total_cases', test_size=0.2, random_state=42):
    """
    Preprocesa los datos, extrae características temporales y divide en conjuntos de entrenamiento y prueba.
    """
    # --- Debugging Prints Start ---
    print(f"--- Inside preprocess_and_split ---")
    print(f"Value of target_col at entry: {target_col}")
    print(f"Type of target_col at entry: {type(target_col)}")
    # --- Debugging Prints End ---

    # Extraer características temporales
    df = df.copy() # Good practice
    df['month'] = df['week_start_date'].dt.month
    df['day'] = df['week_start_date'].dt.day
    df['day_of_week'] = df['week_start_date'].dt.dayofweek

    # Transformar la variable objetivo con log1p
    print(f"Attempting to access df column using target_col: '{target_col}'") # Debug print
    try:
        y = np.log1p(df[target_col])
    except KeyError as e:
        print(f"ERROR: KeyError occurred when accessing df[target_col].")
        print(f"       Value of target_col was: {target_col}")
        print(f"       Type of target_col was: {type(target_col)}")
        print(f"       Columns available in df: {df.columns.tolist()}")
        raise e # Re-raise the error after printing details
    except Exception as e:
        print(f"ERROR: An unexpected error occurred: {e}")
        raise e

    X = df.drop(columns=[target_col, 'week_start_date'])

    # Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True
    )

    # Ajustar y transformar los datos con el preprocesador
    # --- Optional: Use set_output for pandas DataFrame output ---
    if hasattr(preprocessor, 'set_output'):
        try:
            preprocessor.set_output(transform="pandas")
        except Exception as e:
            print(f"Note: Could not set preprocessor output to pandas: {e}")
    # ----------------------------------------------------------

    print("Fitting preprocessor...") # Debug print
    preprocessor.fit(X_train)
    print("Transforming train data...") # Debug print
    X_train_processed = preprocessor.transform(X_train)
    print("Transforming test data...") # Debug print
    X_test_processed = preprocessor.transform(X_test)

    # --- Optional: Verification for NaNs after processing ---
    if isinstance(X_train_processed, pd.DataFrame) and X_train_processed.isnull().sum().sum() > 0:
         print("Advertencia: NaNs detectados en X_train DESPUÉS del preprocesamiento.")
    if isinstance(X_test_processed, pd.DataFrame) and X_test_processed.isnull().sum().sum() > 0:
         print("Advertencia: NaNs detectados en X_test DESPUÉS del preprocesamiento.")
    # -------------------------------------------------------

    print("--- Exiting preprocess_and_split ---") # Debug print
    return X_train_processed, X_test_processed, y_train, y_test



# Define the `train_predict_evaluate` function (MODIFICADA)
def train_predict_evaluate(model_name, search_method, X_train, X_test, y_train, y_test):
    """
    Entrena un modelo especificado usando un método de búsqueda de hiperparámetros,
    realiza predicciones y evalúa el rendimiento.
    """
    print(f"\n--- Entrenando {model_name} usando {search_method} ---")

    # --- Definir Modelos y Espacios de Búsqueda ---
    models = {
        'RandomForest': RandomForestRegressor(random_state=42, n_jobs=-1),
        'KNN': KNeighborsRegressor(n_jobs=-1)
        # Añadir otros modelos aquí si se desea
    }

    # Rejillas para GridSearchCV
    param_grids = {
        'RandomForest': {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, None], # None significa sin límite
            'min_samples_leaf': [3, 5, 10],
            'max_features': ['sqrt', 0.5] # 'sqrt' o una fracción
        },
        'KNN': {
            'n_neighbors': [3, 5, 7, 10],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    }

    # Distribuciones para RandomizedSearchCV
    param_dists = {
        'RandomForest': {
            'n_estimators': randint(100, 301), # Enteros entre 100 y 300
            'max_depth': [10, 15, 20, 25, 30, None], # Lista de opciones
            'min_samples_leaf': randint(1, 11), # Enteros entre 1 y 10
            'max_features': ['sqrt', 'log2', 0.5, 0.7] # Lista de opciones
        },
        'KNN': {
            'n_neighbors': randint(1, 15), # Enteros entre 1 y 14
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan', 'minkowski']
        }
    }

    # --- Seleccionar Modelo y Configurar Búsqueda ---
    if model_name not in models:
        print(f"Error: Modelo '{model_name}' no reconocido.")
        return

    base_model = models[model_name]
    search_cv = None

    if search_method == 'GridSearch':
        if model_name in param_grids:
            search_cv = GridSearchCV(
                estimator=base_model,
                param_grid=param_grids[model_name],
                cv=3, # Número de folds para validación cruzada
                scoring='neg_mean_absolute_error', # MAE negativo (GridSearchCV maximiza)
                verbose=1,
                n_jobs=-1 # Usar todos los cores disponibles
            )
            print(f"Configurando GridSearchCV para {model_name}...")
        else:
            print(f"Advertencia: No hay param_grid definido para {model_name}. Usando modelo base.")
            search_cv = base_model # Usar modelo con parámetros por defecto

    elif search_method == 'RandomSearch':
        if model_name in param_dists:
            search_cv = RandomizedSearchCV(
                estimator=base_model,
                param_distributions=param_dists[model_name],
                n_iter=20, # Número de combinaciones a probar (ajustar según tiempo/recursos)
                cv=3,
                scoring='neg_mean_absolute_error',
                verbose=1,
                n_jobs=-1,
                random_state=42 # Para reproducibilidad
            )
            print(f"Configurando RandomizedSearchCV para {model_name}...")
        else:
            print(f"Advertencia: No hay param_dist definido para {model_name}. Usando modelo base.")
            search_cv = base_model

    elif search_method == 'None':
         print(f"Usando modelo {model_name} con parámetros por defecto.")
         search_cv = base_model # Entrenar directamente sin búsqueda

    else:
        print(f"Error: Método de búsqueda '{search_method}' no reconocido.")
        return

    # --- Entrenar ---
    print("Entrenando...")
    search_cv.fit(X_train, y_train)

    # Obtener el mejor modelo (si se usó búsqueda)
    if hasattr(search_cv, 'best_estimator_'):
        model = search_cv.best_estimator_
        print("Mejores parámetros encontrados:")
        print(search_cv.best_params_)
    else:
        model = search_cv # Caso 'None' o si no había grid/dist

    # --- Predecir y Evaluar ---
    print("Realizando predicciones...")
    predictions_log = model.predict(X_test)

    # Revertir la transformación logarítmica y asegurar no negativos/enteros
    # Es crucial revertir tanto las predicciones como y_test para comparar en la escala original
    predictions = np.maximum(0, np.expm1(predictions_log).round().astype(int))
    y_test_original = np.expm1(y_test) # Revertir y_test también

    # Calcular y mostrar el MAE en la escala original
    mae = mean_absolute_error(y_test_original, predictions)
    print(f'\nMean Absolute Error (MAE) - {model_name} ({search_method}): {mae:.4f}')

    # Mostrar estadísticas del conjunto de prueba en la escala original
    stats = pd.Series(y_test_original).describe()
    print(f"\nEstadísticas de 'total_cases' en el conjunto de prueba (escala original):")
    # Imprimir estadísticas de forma más legible
    print(f"- Media:      {stats['mean']:.4f}")
    print(f"- Mediana:    {stats['50%']:.4f}")
    print(f"- Desv. Est.: {stats['std']:.4f}")
    print(f"- Mínimo:     {stats['min']:.4f}")
    print(f"- Máximo:     {stats['max']:.4f}")

    if stats['mean'] > 0:
        mae_perc_mean = (mae / stats['mean']) * 100
        print(f"\n- MAE como % de la Media: {mae_perc_mean:.2f}%")
    else:
        print("\n- No se puede calcular MAE como % de la Media (la media es 0).")
    print("-" * 40)



# Main Workflow - Load and Merge Data
Load the datasets using the `load_and_merge_data` function and display the merged DataFrame.

In [35]:

FEATURES_FILE = 'dengue_features_train.csv'
LABELS_FILE = 'dengue_labels_train.csv'

# 1. Cargar y unir los datos
# En la celda donde llamas a preprocess_and_split (ej. Celda 24)

merged_df = load_and_merge_data(FEATURES_FILE, LABELS_FILE)



# Main Workflow - Create Preprocessor
Create the preprocessor using the `create_preprocessor` function and display its configuration.

# Main Workflow - Train, Predict, and Evaluate
Train the model, make predictions, and evaluate its performance using the `train_predict_evaluate` function.

In [36]:
if merged_df is not None:
    # 1. Crear el preprocesador ANTES de llamar a preprocess_and_split
    #    Asegúrate de que las columnas usadas aquí coincidan con las que tendrá X dentro de la función
    preprocessor = create_preprocessor(merged_df.drop(columns=['total_cases', 'week_start_date']))
    print("Preprocessor created.") # Mensaje de confirmación

    # 2. Preprocesar y dividir los datos, pasando el preprocesador
    try:
        # Ahora pasamos 'preprocessor' como segundo argumento
        X_train, X_test, y_train, y_test = preprocess_and_split(merged_df, preprocessor)

        if X_train is not None: # Verificar que el preprocesamiento fue exitoso
            print(f"X_train shape: {X_train.shape}") # Imprimir shapes para verificar

            # 3. Entrenar, predecir y evaluar diferentes modelos y métodos de búsqueda
            
            train_predict_evaluate('RandomForest', 'GridSearch', X_train, X_test, y_train, y_test)
            train_predict_evaluate('RandomForest', 'RandomSearch', X_train, X_test, y_train, y_test)
            train_predict_evaluate('KNN', 'GridSearch', X_train, X_test, y_train, y_test)
            train_predict_evaluate('KNN', 'RandomSearch', X_train, X_test, y_train, y_test)

        else:
            print("Error durante el preprocesamiento y división de datos.")

    except Exception as e:
        print(f"ERROR during preprocess_and_split call or subsequent steps: {e}")
        import traceback
        traceback.print_exc()

else:
    print("Proceso detenido debido a error en la carga de archivos.")

Preprocessor created.
--- Inside preprocess_and_split ---
Value of target_col at entry: total_cases
Type of target_col at entry: <class 'str'>
Attempting to access df column using target_col: 'total_cases'
Fitting preprocessor...
Transforming train data...
Transforming test data...
--- Exiting preprocess_and_split ---
X_train shape: (1164, 27)

--- Entrenando RandomForest usando GridSearch ---
Configurando GridSearchCV para RandomForest...
Entrenando...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


c:\Users\Javier\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros encontrados:
{'max_depth': None, 'max_features': 0.5, 'min_samples_leaf': 3, 'n_estimators': 100}
Realizando predicciones...

Mean Absolute Error (MAE) - RandomForest (GridSearch): 11.6199

Estadísticas de 'total_cases' en el conjunto de prueba (escala original):
- Media:      25.4829
- Mediana:    13.0000
- Desv. Est.: 41.7328
- Mínimo:     0.0000
- Máximo:     410.0000

- MAE como % de la Media: 45.60%
----------------------------------------

--- Entrenando RandomForest usando RandomSearch ---
Configurando RandomizedSearchCV para RandomForest...
Entrenando...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros encontrados:
{'max_depth': 10, 'max_features': 0.7, 'min_samples_leaf': 2, 'n_estimators': 108}
Realizando predicciones...

Mean Absolute Error (MAE) - RandomForest (RandomSearch): 11.1747

Estadísticas de 'total_cases' en el conjunto de prueba (escala original):
- Media:      25.4829
- Mediana:    13.0000
- Desv. Est.: 41.7328
- 